<a href="https://colab.research.google.com/github/johnnybigoode/practical-time-series/blob/main/TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preface

Starts with examples of things that form time series, like weather, stock markets and heartbeats.

Reminds us of the future of IoT and the amount of sensors and gadgets available for high-quality time series data

Expects a introductory statistics background

There are at least two youtube 3h long scipy talks with the author on the subject

https://www.youtube.com/watch?v=JNfxr4BQrLk

https://www.youtube.com/watch?v=v5ijNXvlC5A&t=1s

book errata
https://www.oreilly.com/catalog/errata.csp?isbn=0636920187714

github repo
https://github.com/PracticalTimeSeriesAnalysis/BookRepo

# Chapter 1 Time Series: Overview and Quick History

Time series: Data in choronological order

Time series analysis: Diagnose the past and predict the future

Medical field examples:

electrocardiograms - records electrig signals passing thru the heart
electroencephalogram - measures eletrical impulses in the brain

examples of time series in weather:

barometer - taking meassurements of the state of the atmosphere at regular invervals

examples of time series in bussiness:

forecasting demand, estimaging future raw materials prices, hedging on manufacturing costs

examples of modern machine learning related to time-series:

computer security, anomaly detection as a method for getting hackers and invader dynamic time warping, a way of checking similarity of time series
recursive neural networks show usefulness for extracting patterns in corrupted data